In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from features.settings import Baseline_Settings, Pair_Hinge_Settings, Pair_Log_Settings, Triplet_Hinge_Settings, Triplet_Log_Settings
from features.models import ConvModelSmall, ConvModelMedium, ConvModelLarge
from features.models import GCNNModelSmall, GCNNModelMedium, GCNNModelLarge
from classes.models import LinearClassifier
from datasets import load_cifar100
from sklearn.svm import LinearSVC
import time

/home/dqi/.conda/envs/py36/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/dqi/.conda/envs/py36/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data, samplers, pair_samplers, triplet_samplers = load_cifar100(augment=True)

CIFAR-100 loaded in 103 seconds


In [3]:
def test_model(feature_model, settings, data, num_features, num_classes, feature_samplers, classification_samplers,
               f_num_steps, f_lr, f_keep_prob, f_reg, c_num_steps, c_lr, c_reg):
    
    tf.reset_default_graph()
    
    f_train, f_valid, f_test = feature_samplers
    c_train, c_valid, c_test = classification_samplers
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = data
    _, H, W, C = x_train.shape
    
    x = tf.placeholder(tf.float32, shape=[None, H, W, C])
    y = tf.placeholder(tf.int64, shape=[None])    
    dropout = tf.placeholder(tf.float32)
    training = tf.placeholder(tf.bool)
    f_model = feature_model(x=x, y=y, settings=settings, num_chan=C, num_features=num_features, 
              lr=f_lr, reg=f_reg, dropout=dropout, training=training)
    
    features = tf.placeholder(tf.float32, shape=[None, num_features])
    c_model = LinearClassifier(x=features, y=y, num_features=num_features, num_classes=num_features, 
                               lr=c_lr, reg=c_reg)
    
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        print("begin training features // num_features: %g, lr: %g, reg: %g, dropout: %g" %(num_features, f_lr, f_reg, f_keep_prob))
        train_time = time.time()
        for step in range(f_num_steps):        
            x_, y_ = f_train.sample(700)
            sess.run(f_model.optimize, feed_dict={x:x_, y:y_, dropout:f_keep_prob, training:True})     
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([f_model.loss, f_model.acc], 
                                                 feed_dict={x:x_, y:y_, dropout:1.0, training:False})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training features // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        x_, y_ = f_valid.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        x_, y_ = f_test.sample(1000)
        test_error = 1 - sess.run(f_model.acc, feed_dict={x:x_, y:y_, dropout:1.0, training:False})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        print("begin training classifier")
        train_time = time.time()
        for step in range(c_num_steps):        
            x_, y_ = c_train.sample(700)
            features_ = sess.run(f_model.features, feed_dict={x:x_, dropout:1.0, training:False})
            sess.run(c_model.optimize, feed_dict={features:features_, y:y_, x:x_, dropout:1.0, training:False})
            if step % 1000 == 0:
                train_loss, train_acc = sess.run([c_model.loss, c_model.acc], feed_dict={features:features_, y:y_})
                print("\tstep %d: train loss %g, train error %g"%(step, train_loss, 1 - train_acc))  
        train_time = time.time() - train_time
        print("end training classifier // time elapsed: %.4f s"%(train_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_valid, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_valid})
        eval_test_time = time.time() - eval_test_time
        print("validation set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))
        
        eval_test_time = time.time()
        features_ = sess.run(f_model.features, feed_dict={x:x_test, dropout:1.0, training:False})
        test_error = 1 - sess.run(c_model.acc, feed_dict={features:features_, y:y_test})
        eval_test_time = time.time() - eval_test_time
        print("test set error: %.4f // time elapsed: %.4f s"%(test_error, eval_test_time))

In [4]:
test_model(feature_model=GCNNModelSmall, settings=Baseline_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.001, reg: 0.001, dropout: 0.7
	step 0: train loss 5.17647, train error 0.997143
	step 1000: train loss 5.00519, train error 0.957143
	step 2000: train loss 4.95009, train error 0.961429
	step 3000: train loss 4.90154, train error 0.941429
	step 4000: train loss 4.80595, train error 0.935714
	step 5000: train loss 4.80295, train error 0.94
	step 6000: train loss 4.66479, train error 0.897143
	step 7000: train loss 4.6549, train error 0.912857
	step 8000: train loss 4.55748, train error 0.907143
	step 9000: train loss 4.56257, train error 0.9
	step 10000: train loss 4.55824, train error 0.901429
	step 11000: train loss 4.51331, train error 0.908571
	step 12000: train loss 4.48371, train error 0.885714
	step 13000: train loss 4.40715, train error 0.885714
	step 14000: train loss 4.37365, train error 0.861429
	step 15000: train loss 4.38407, train error 0.864286
	step 16000: train loss 4.36995, train error 0.867143
	step 17000: train loss

In [5]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)

begin training features // num_features: 100, lr: 0.001, reg: 0.001, dropout: 0.7
	step 0: train loss 1.53548, train error 0.478571
	step 1000: train loss 1.47141, train error 0.452857
	step 2000: train loss 1.44291, train error 0.398571
	step 3000: train loss 1.48774, train error 0.452857
	step 4000: train loss 1.48073, train error 0.45
	step 5000: train loss 1.45626, train error 0.447143
	step 6000: train loss 1.48187, train error 0.434286
	step 7000: train loss 1.45791, train error 0.427143
	step 8000: train loss 1.46465, train error 0.46
	step 9000: train loss 1.44453, train error 0.431429
	step 10000: train loss 1.38156, train error 0.411429
	step 11000: train loss 1.348, train error 0.41
	step 12000: train loss 1.38945, train error 0.42
	step 13000: train loss 1.37956, train error 0.412857
	step 14000: train loss 1.37284, train error 0.407143
	step 15000: train loss 1.35174, train error 0.384286
	step 16000: train loss 1.39454, train error 0.402857
	step 17000: train loss 1.3183,

In [6]:
test_model(feature_model=GCNNModelSmall, settings=Triplet_Hinge_Settings, num_features=100, num_classes=100,
           data=data, feature_samplers=triplet_samplers, classification_samplers=samplers,           
           f_num_steps=70000, f_lr=0.005, f_keep_prob=0.6, f_reg=0.001, 
           c_num_steps=30000, c_lr=0.005, c_reg=0.01)